In [1]:
pip install -r ../../requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, get_data_commitment_maps

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

Here is a weird example where different col1 and col2 of data.json have different row number. We just want to show that different dimension is possible. In fact, a person can just request median(col_1), and median(col_2). Then just compute mean on his own as well, but here we show that the code is composable enough to do all at once. 

In [5]:

data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')
create_dummy(data_path, dummy_data_path)


In [6]:
scales = [8]
selected_columns = ['col_1', 'col_2']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [7]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x1, x2 = data[0], data[1]
    out_0 = s.median(x1)
    out_1 = s.median(x2)
    return s.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))

error = 0.0001
_, verifier_model = computation_to_model(computation, error)

verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path, verifier_model, verifier_model_path)

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_36701/589984064.py:9: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return s.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_36701/589984064.py:9: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  return s.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))
/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:166: TracerWarning: torch.tensor results are registered as constants in the trace. You ca

In [8]:
# prover calculates settings, send to verifier
_, prover_model = computation_to_model(computation, error)

prover_gen_settings(data_path,selected_columns, sel_data_path, prover_model, prover_model_path, scales, "resources", settings_path)

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_36701/589984064.py:9: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return s.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_36701/589984064.py:9: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  return s.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))


==== Generate & Calibrate Setting ====
scale:  [8]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":8,"param_scale":8,"scale_rebase_multiplier":10,"lookup_range":[-25754,25754],"logrows":16,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":20992,"total_assignments":10073,"total_const_size":14,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,8],"model_input_scales":[8,8],"module_sizes":{"kzg":[],"poseidon":[20992,[2]],"elgamal":[0,[0]]},"required_lookups":[{"GreaterThan":{"a":0.0}},"KroneckerDelta"],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


In [9]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 6.3793909549713135 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 49.3515625
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[3042937791208075219, 8157070662846698822, 3804781648660056856, 172406108020799675], [15295097400487804665, 12861486368330479023, 3350118022201779210, 343142782800691716], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [10870267098303494893, 1752989342377741058, 8860763459400202009, 2635465469930673149]]], 'proof': '01017184353e695aced6d13780734e00fadcb43c7ee8131caae1cac20b8184741c7a2e011a6cc9b8ad0a16b82b652215f1cbf4b9d23d39893fbd5ca3350aa6b925e9c8af2592dec9cb4ee60abc3aff54e4779b665686d08547e0614492575a9922d62f6df72af149659230e804f5e3943c51d670bd65c272766f198140035f25026e0807afbf2b18f11517427f908e534cf4567ac61996d1c1a8b83e790e659721f1a44f7e6bd5942228c14a6e72ea6e51ce6b540606139c5abc66039100941517f66d072251b99aed84dd8b9a43065bbf4b1ddfb00d07868cb68448d4d7ee9f16730cee66adbaa816108da226f713aa1a167f587bd88d14ddee2fcef8ccc7490b11856885436ddf8c77c28ba0c6a7258953c7de6012c40c7fc5540f66b462a5108d65c44b3772f4bf0427978be3cc2d6892779023

In [10]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)
print("Verifier gets result:", res)

Verifier gets result: 49.3515625
